In [2]:
import sys
sys.path

['',
 'C:\\WINDOWS\\SYSTEM32\\python27.zip',
 'c:\\python27\\DLLs',
 'c:\\python27\\lib',
 'c:\\python27\\lib\\plat-win',
 'c:\\python27\\lib\\lib-tk',
 'c:\\python27',
 'c:\\python27\\lib\\site-packages',
 'C:\\ProgramData\\Ableton\\Live 10 Suite\\Resources\\MIDI Remote Scripts',
 'C:\\Users\\thiba\\Google Drive\\music\\dev\\toto2',
 'c:\\python27\\lib\\site-packages\\win32',
 'c:\\python27\\lib\\site-packages\\win32\\lib',
 'c:\\python27\\lib\\site-packages\\Pythonwin',
 'c:\\python27\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\thiba\\.ipython']

In [1]:
import openapi_client
print(openapi_client.__file__)
client = openapi_client.ApiClient()
print(client.configuration.retries)

c:\python27\lib\site-packages\openapi_client\__init__.pyc
2
